In [1]:
import asyncio
import gzip
import json
import logging
import os
import sys
import urllib
import warnings
from datetime import datetime

import dotenv
import pandas as pd
import requests
from sqlalchemy import create_engine, text

In [2]:
import importlib.util

path = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_model.py"
spec = importlib.util.spec_from_file_location("rerun_model", path)
rerun_model = importlib.util.module_from_spec(spec)
spec.loader.exec_module(rerun_model)

print("Model version:", rerun_model.get_model_version())


Model version: 16.15.4


#### See if IBVStatus Data is ready

In [3]:
import pandas as pd

csv_path = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_shadow_ibvV1.csv"

df_ibv = pd.read_csv(csv_path)

if "IBVStatusID" not in df_ibv.columns:
    raise ValueError("CSV missing 'IBVStatusID' column")

total_rows = len(df_ibv)
unique_ids = df_ibv["IBVStatusID"].nunique(dropna=True)
duplicate_count = total_rows - unique_ids

print(f"Total rows: {total_rows}")
print(f"Unique IBVStatusID: {unique_ids}")
print(f"Duplicate count: {duplicate_count}")

# IDs appearing more than once (including NaN if present)
vc = df_ibv["IBVStatusID"].value_counts(dropna=False)
dupe_ids = vc[vc > 1]
print("\nTop duplicate IDs (count > 1), first 20:")
print(dupe_ids.head(20))

# Show sample rows of duplicates
dupes_df = df_ibv[df_ibv["IBVStatusID"].isin(dupe_ids.index)].sort_values("IBVStatusID")
dupes_df.head(20)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_shadow_ibvV1.csv'

In [9]:
if 'DateProcessed' in df_ibv.columns:
    df_ibv['DateProcessed'] = pd.to_datetime(df_ibv['DateProcessed'])
    df_ibv_sorted = df_ibv.sort_values(["IBVStatusID", "DateProcessed"]).drop_duplicates("IBVStatusID", keep="last")
print(len(df_ibv_sorted)) 

6030


In [10]:
df_ibv_sorted.head()

,CModelLogID,ModelName,ApplicationID,PortfolioID,IBVStatusID,DateProcessed,ModelRequest,ModelResponse,ProcessingTime
380,1315,ibv,0,0,757,2025-05-10 12:42:07.290,"{""asOfDate"":""2025-05-10"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""ErqbnE1VRNhYZzm0r...",3031
4147,5456,ibv,0,0,1258,2025-08-27 08:51:35.213,"{""asOfDate"":""2025-08-27"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""Z1y3oybRZwsyz97P1...",1563
3336,4645,ibv,0,0,1382,2025-08-05 11:45:12.307,"{""asOfDate"":""2025-08-05"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""639pvqA83piYK5gjM...",2546
639,1756,ibv,0,0,1810,2025-05-22 09:35:23.653,"{""asOfDate"":""2025-05-22"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""AeRBY75BDwUm78Mon...",1562
1533,2838,ibv,0,0,2334,2025-06-24 12:28:42.160,"{""asOfDate"":""2025-06-24"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""5PeDy5EZ9mtLb6kQD...",2203


#### Save IBV Status to CSV for env file DATAPATH

In [11]:
df_deduped = df_ibv_sorted[['IBVStatusID','DateProcessed']].rename(columns={"DateProcessed": "DateCreated"})
deduped_csv = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_ibv_shadowV1_dedup.csv"
df_deduped.to_csv(deduped_csv, index=False)